# Red Light Cameras and Crashes

Do crashes and injuries go down at intersections where red light cameras were installed?

In [1]:
import folium
import datetime
import pandas as pd
from sqlalchemy import text
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
# include crashes that occurred within this distance of the given latitude and longitude
radius_miles = 0.1

In [4]:
def crashes_at_location(location_str):
    
    lat_lon = cda.geocode_location(location_str + ', Denver, CO')
    
    query = f"""
    select
    count(*) as num_crashes
    , sum((driver_action like '%failed to stop at signal%')::int) as red_light_crashes
    , sum(fatality::int) as fatalities
    , sum(sbi::int) as sbi
    , sum(pedestrian_ind::int) as pedestrian_ind
    , sum(bicycle_ind::int) as bicycle_ind
    
    from crashes

    where geo_lon is not null
    and ST_Distance(
        geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
        ) < {radius_miles * 5280}
    """
    
    nearby = pd.read_sql(text(query), cda.conn)
    
    return nearby

In [5]:
locations_dict = {
    '6th & Kalamath': True
    , '6th Ave & Lincoln St': True
    , '8th & Speer': True
    , '36th & Quebec': True
    , 'East 13th Avenue and Lincoln Street': False
    , 'East 18th Avenue and Lincoln': False
    , 'West Alameda Avenue and Santa Fe Drive': False

}

locations = pd.DataFrame.from_dict(
    locations_dict, orient='index', columns=['has_red_light_camera']
).reset_index(
    names='address'
)

In [6]:
for idx, row in locations.iterrows():
    temp = crashes_at_location(row['address'])
    
    for f in ['num_crashes', 'red_light_crashes', 'fatalities', 'sbi', 'pedestrian_ind', 'bicycle_ind']:
        locations.loc[idx, f] = temp.loc[0, f]

In [7]:
cda.conn.dispose()

## Crashes involving Red Light Running

What percentage of crashes at each intersection have "failed to stop at signal" listed as one of the contributing driver actions in the crash? 

In [8]:
locations['perc_red_light'] = locations['red_light_crashes'] / locations['num_crashes']
locations.sort_values(by='perc_red_light')

,address,has_red_light_camera,num_crashes,red_light_crashes,fatalities,sbi,pedestrian_ind,bicycle_ind,perc_red_light
6,West Alameda Avenue and Santa Fe Drive,False,1720.0,56.0,1.0,15.0,16.0,10.0,0.032558
3,36th & Quebec,True,464.0,28.0,1.0,7.0,19.0,3.0,0.060345
0,6th & Kalamath,True,534.0,40.0,1.0,5.0,7.0,0.0,0.074906
4,East 13th Avenue and Lincoln Street,False,442.0,37.0,2.0,10.0,29.0,4.0,0.083710
1,6th Ave & Lincoln St,True,1016.0,105.0,1.0,13.0,36.0,19.0,0.103346
5,East 18th Avenue and Lincoln,False,592.0,90.0,0.0,3.0,16.0,13.0,0.152027
2,8th & Speer,True,562.0,116.0,0.0,8.0,12.0,11.0,0.206406


In [9]:
locations.groupby('has_red_light_camera')[locations.select_dtypes(include='number').columns].mean()

,num_crashes,red_light_crashes,fatalities,sbi,pedestrian_ind,bicycle_ind,perc_red_light
has_red_light_camera,,,,,,,
False,918.0,61.00,1.00,9.333333,20.333333,9.00,0.089432
True,644.0,72.25,0.75,8.250000,18.500000,8.25,0.111251
